### 1. Tạo pertured data và lưu giống data cũ (cal_metric.py)
    1. Tạo ra câu pertured va convert json và ghép label (pertured_folder/masked_data_json)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. đưa json (masked data 0.1) vào model lấy logit, prediction -> cần tạo class để lấy cho cả origin data và masked data
### 3. convert label gold theo từ thành label gold theo token
### 4. Tính inf 

In [2]:
import json
import os
import torch
import sys
from tqdm import tqdm
from mlm_utils.model_utils import TOKENIZER
from mlm_utils.transform_func import get_files, encode_text
from mlm_utils.pertured_dataset import PerturedDataset
from pathlib import Path


## Step 2

### 2.3


In [1]:
import json
import os
import torch
import sys
from tqdm import tqdm
from mlm_utils.model_utils import TOKENIZER
from mlm_utils.transform_func import get_files, encode_text
from mlm_utils.pertured_dataset import PerturedDataset
from pathlib import Path

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-18 00:44:03.393636: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-18 00:44:03.488413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 00:44:05.390236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-18 00:44:06.879489: E tensorflow/compiler/xla/stream_executor/cuda/cu

In [2]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_params(model_file):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Load finetuned model
    loadedDict = torch.load(model_file, map_location=torch.device(device))

    taskParams = loadedDict['task_params']
   
    allParams = {}
    allParams['task_params'] = taskParams
    allParams['gpu'] = torch.cuda.is_available()

    # dummy values
    allParams['num_train_steps'] = 10
    allParams['warmup_steps'] = 0
    allParams['learning_rate'] = 2e-5
    allParams['epsilon'] = 1e-8

    return allParams, loadedDict      

In [3]:
import torch.nn as nn
import numpy as np
sys.path.append('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/')
from SRL.model import multiTaskModel
model_file = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/output/multi_task_model_9_13050.pt')
allParams, loadedDict = load_params(model_file)
model = multiTaskModel(allParams)
model.load_multi_task_model(loadedDict)


In [6]:
import pandas as pd

def get_pred_score(dataloader, model):
    allPreds = []
    allScores = []
    allLogitsSoftmax = []
    allLogitsRaw = []
    allLabels = []
    model.network.eval()
    for batch in tqdm(dataloader, total = len(dataloader)):
        batch = tuple(t.to(device) if isinstance(t, torch.Tensor) else t for t in batch)
      
        origin_uid, origin_id, label, token_id, type_id, mask = batch
        with torch.no_grad():
            outputs_model, logits = model.network(token_id, type_id, mask, 0, 'conllsrl')
           
            outLogitsSoftmax = nn.functional.softmax(logits, dim = 2).data.cpu().numpy()
            
            outLogitsSigmoid = nn.functional.sigmoid(logits).data.cpu().numpy()
            
            predicted_sm = np.argmax(outLogitsSoftmax, axis = 2)
            
            # here in score, we only want to give out the score of the class of tag, which is maximum
            predScore = np.max(outLogitsSigmoid, axis = 2).tolist() 
            
            predicted_sm = predicted_sm.tolist()
        
            # get the attention masks, we need to discard the predictions made for extra padding
            predictedTags = []
            predScoreTags = []
            
            # if mask is not None:
            #     #shape of attention Masks (batchSize, maxSeqLen)
            #     actualLengths = mask.cpu().numpy().sum(axis = 1).tolist()
            
            #     for i, (pred, sc) in enumerate(zip(predicted_sm, predScore)):
            #         predictedTags.append( pred[:actualLengths[i]] )
            #         predScoreTags.append( sc[:actualLengths[i]])
    
            # else:
            #     predictedTags = predicted_sm
            #     predScoreTags = predScore
            
            # để nguyên kh cần lấy theo attention mask
            allPreds.append(predicted_sm)  
            allScores.append(predScore)  
            allLogitsSoftmax.append(outLogitsSoftmax)
            allLabels.append(label.tolist())
            allLogitsRaw.append(logits.data.cpu().numpy())
        break
    # flatten allPreds, allScores
    allPreds = [item for sublist in allPreds for item in sublist]
    allScores = [item for sublist in allScores for item in sublist]
    allLabels = [item for sublist in allLabels for item in sublist]
    allLogitsSoftmax = [item for sublist in allLogitsSoftmax for item in sublist]
    allLogitsRaw = [item for sublist in allLogitsRaw for item in sublist]
    return allPreds, allScores, allLabels, allLogitsSoftmax, allLogitsRaw      

      
    
def write_pred_label(allPreds, allScores, allLabels, allParams, wrtFile):
    
    allLabels = []
    allIds = []
    
    labMap = allParams['task_params'].labelMap['conllsrl']
    
    labMapRevN = {v:k for k,v in labMap.items()}
   
    for j, (p, l) in enumerate(zip(allPreds, allLabels)):
        allLabels[j] = l[:len(p)]
        allPreds[j] = [labMapRevN[int(ele)] for ele in p]
        allLabels[j] = [labMapRevN[int(ele)] for ele in allLabels[j]]
    # allPreds[i] = [ [ labMapRev[int(p)] for p in pp ] for pp in allPreds[i] ]
    # allLabels[i] = [ [labMapRev[int(l)] for l in ll] for ll in allLabels[i] ]

    newPreds = []
    newLabels = []
    newScores = []
    for m, samp in enumerate(allLabels):
        Preds = []
        Labels = []
        Scores = []
        for n, ele in enumerate(samp):
            #print(ele)
            if ele != '[CLS]' and ele != '[SEP]' and ele != 'X':
                #print('inside')
                Preds.append(allPreds[m][n])
                Labels.append(ele)
                Scores.append(allScores[m][n])
                
        newPreds.append(Preds)
        newLabels.append(Labels)
        newScores.append(Scores)
    
    allLabels = newLabels
    allPreds = newPreds
    allScores = newScores
    df = pd.DataFrame({"uid" : allIds, "prediction" : allPreds, "label" : allLabels})
    
    df.to_csv(wrtFile, sep = "\t", index = False)
    

In [7]:
# nếu số kh cao thì lấy folder masked_data_json_v2
writeDir = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/pertured_data/masked_data_json_v2')
dataset = PerturedDataset(
        file_name= os.path.join(writeDir,'pertured_data_alter_full.json'),
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

dataloader = dataset.generate_batches(
            dataset= dataset,
            batch_size=32)
allPreds, allScores,allLabels, allLogitsSoftmax, allLogitsRaw  = get_pred_score(dataloader, model)
wrtFile = "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/pertured_data_prediction.json"


  0%|          | 0/49 [00:06<?, ?it/s]


In [9]:
print(len(allPreds[0]))
print(len(allLabels[0]))

81
85


In [ ]:
{'B-A1': 0,
 'I-A1': 1,
 'O': 2,
 'B-V': 3,
 'B-A0': 4,
 'I-A0': 5,
 'B-A4': 6,
 'I-A4': 7,
 'I-A3': 8,
 'B-A2': 9,
 'I-A2': 10,
 'B-A3': 11,
 '[CLS]': 12,
 '[SEP]': 13,
 'X': 14}

In [ ]:
def get_idx_arg_preds(preds_origin, preds_masked, label_origin=None): # label_origin: nhãn gold
    list_idx_arg_change = []
    assert len(preds_origin) == len(preds_masked), 'Length of preds_origin and preds_masked must be the same'
    for i in range(len(preds_origin)):
        test = preds_origin[i] not in [2, 3, 12, 13, 14] and preds_masked[i] not in [2, 3, 12, 13, 14]
        if label_origin:
            assert len(preds_origin) == len(label_origin), 'Length of preds_origin and label_origin must be the same'
            if test or label_origin[i] not in [2, 3, 12, 13, 14]:
                list_idx_arg_change.append(i)
        else:
            if test:
                list_idx_arg_change.append(i)
    return list_idx_arg_change

def calculateInfluenceScore(outLogitsSigmoid_original, outLogitsSigmoid_meddle, list_arg_change):
    influence_score = []
    weight = []
    assert len(outLogitsSigmoid_original) == len(outLogitsSigmoid_meddle) 
    for i in range(len(outLogitsSigmoid_original)):
        if i not in list_arg_change:
            continue
        max_index_original = torch.argmax(outLogitsSigmoid_original[i])
        max_index_meddle = torch.argmax(outLogitsSigmoid_meddle[i])
        if max_index_original == max_index_meddle:
            influence_score.append((outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_meddle]))
            weight.append(1)
        else:
            influ_old_label = (outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_original]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_original])
            influ_new_label = (outLogitsSigmoid_meddle[i][max_index_meddle] - outLogitsSigmoid_original[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_meddle], outLogitsSigmoid_meddle[i][max_index_meddle])
            influence_score.append(influ_old_label + influ_new_label)
            weight.append(2)
    return influence_score, weight

In [10]:
import torch
for i in range(len(resultwordOrigin['logits'])):
    resultwordOrigin['logits'][i] = torch.tensor(resultwordOrigin['logits'][i])
for i in range(len(resultwordMasked['logits'])):
    resultwordMasked['logits'][i] = torch.tensor(resultwordMasked['logits'][i])
    
influenceScore, weight = calculateInfluenceScore(resultwordOrigin['logits'][2], resultwordMasked['logits'][3], get_idx_arg_preds(resultwordOrigin['preds'][2], resultwordMasked['preds'][3]))

## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [ ]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 85